In [3]:
import pandas as pd
orders = pd.read_csv(r"C:\Users\Hp\Downloads\orders.csv")


In [4]:
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [5]:
users = pd.read_json(r"C:\Users\Hp\Downloads\users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [6]:
import sqlite3

conn = sqlite3.connect("food.db")

with open(r"C:\Users\Hp\Downloads\restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [7]:
orders.columns
users.columns
restaurants.columns


Index(['restaurant_id', 'restaurant_name', 'cuisine', 'rating'], dtype='object')

In [8]:
orders.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name'],
      dtype='object')

In [9]:
users.columns

Index(['user_id', 'name', 'city', 'membership'], dtype='object')

In [10]:
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)

final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)


In [11]:
final_df = final_df.drop(columns=["restaurant_name_y"])
final_df = final_df.rename(columns={"restaurant_name_x": "restaurant_name"})


In [13]:
final_df.to_csv(
    r"C:\Users\Hp\Downloads\final_food_delivery_dataset.csv",
    index=False
)


In [14]:
final_df.shape


(10000, 11)

In [15]:
final_df.head()



,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Italian,3.0


In [16]:
final_df.isnull().sum()


order_id           0
user_id            0
restaurant_id      0
order_date         0
total_amount       0
restaurant_name    0
name               0
city               0
membership         0
cuisine            0
rating             0
dtype: int64

In [17]:
final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [18]:
final_df.groupby("cuisine")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [19]:
high_value_users = (
    final_df
    .groupby("user_id")["total_amount"]
    .sum()
    .reset_index()
)

count_users = high_value_users[high_value_users["total_amount"] > 1000]["user_id"].nunique()

count_users


2544

In [20]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0 – 3.5", "3.6 – 4.0", "4.1 – 4.5", "4.6 – 5.0"]

final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=bins,
    labels=labels,
    include_lowest=True
)


In [21]:
final_df.groupby("rating_range")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)



C:\Users\Hp\AppData\Local\Temp\ipykernel_10016\282105964.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"] \


rating_range
4.6 – 5.0    2197030.75
3.0 – 3.5    2136772.70
4.1 – 4.5    1960326.26
3.6 – 4.0    1717494.41
Name: total_amount, dtype: float64

In [22]:
final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)



city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [23]:
cuisine_stats = final_df.groupby("cuisine").agg(
    distinct_restaurants=("restaurant_id", "nunique"),
    total_revenue=("total_amount", "sum")
).sort_values("distinct_restaurants")

cuisine_stats



,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [24]:
gold_orders = final_df[final_df["membership"] == "Gold"]["order_id"].nunique()
total_orders = final_df["order_id"].nunique()

percentage = round((gold_orders / total_orders) * 100)
percentage



50

In [25]:
restaurant_stats = final_df.groupby("restaurant_name").agg(
    total_orders=("order_id", "nunique"),
    avg_order_value=("total_amount", "mean")
)

filtered = restaurant_stats[restaurant_stats["total_orders"] < 20] \
    .sort_values("avg_order_value", ascending=False)

filtered.head()


,total_orders,avg_order_value
restaurant_name,,
Hotel Dhaba Multicuisine,13,1040.222308
Sri Mess Punjabi,12,1029.180833
Ruchi Biryani Punjabi,16,1002.140625
Sri Delights Pure Veg,18,989.467222
Classic Kitchen Family Restaurant,19,973.167895


In [26]:
final_df.groupby(["membership", "cuisine"])["total_amount"] \
    .sum() \
    .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [27]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

final_df.groupby("quarter")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)


C:\Users\Hp\AppData\Local\Temp\ipykernel_10016\3064641765.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [29]:
gold_order_count=final_df[final_df["membership"]=="Gold"]["order_id"].nunique()
gold_order_count

4987

In [30]:
hyderabad_revenue = ( final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum())

round(hyderabad_revenue)


1889367

In [31]:
distinct_users = final_df["user_id"].nunique()
distinct_users


2883

In [32]:
avg_or=final_df[final_df["membership"]=="Gold"]["total_amount"].mean()
round(avg_or,2)

np.float64(797.15)

In [42]:
high_ri= final_df["order_id".count()]["rating"]>=4.5
high_ri

TypeError: count expected at least 1 argument, got 0

In [43]:
high_ri_count = final_df[final_df["rating"] >= 4.5]["order_id"].nunique()
high_ri_count


3374

In [44]:
gold_city_revenue = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [45]:
top_city = gold_city_revenue.index[0]
top_city


'Chennai'

In [46]:
orders_in_top_city = (
    final_df[
        (final_df["membership"] == "Gold") &
        (final_df["city"] == top_city)
    ]["order_id"]
    .nunique()
)

orders_in_top_city


1337

In [47]:
len(final_df)


10000